# Customers Churn from a subscription to a service

## (Telecom Dataset)

# 1. Read Data and Explore

In this approach, we will analyze datasets to summarize their main characteristics, with visual and quantitative methods.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn import metrics, model_selection
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier 


def evalmodel(y_predicted, y_pred_prob, y_actual, modelname=None, dataset=None):
    cm = confusion_matrix(y_predicted, y_actual)
    TP = cm[1,1]
    TN = cm[0,0]
    FP = cm[1,0]
    FN = cm[0,1]
    accuracy = (TP+TN)/(TP+FP+TN+FN)
    sensitivity = TP/(TP+FN)
    specificity = TN/(TN+FP)
    fpr, tpr, _ = roc_curve(y_actual, y_pred_prob)
    AUC_value = round(auc(fpr, tpr),4)
    return(pd.DataFrame({'modelname':[modelname],'dataset':[dataset],'accuracy':[accuracy], 'sensitivity':[sensitivity], 'specificity':[specificity], 'AUC':[AUC_value]}))

%matplotlib inline

### 1.1 Load Data


In [ ]:
# load csv
customers = pd.read_csv('churn.csv')

# remove target and ID columns
features = customers.drop(columns=['area_code', 'phone_number', 'class'])

# target
targets = customers['class']

In [ ]:
targets.value_counts()

In [ ]:
features

### 1.2 Univariate Analysis

In [ ]:
# describe the characteristics of variables
features.describe()

### 1.3 Correlation matrix

A correlation matrix describing the correlation between the different features in the dataset.

In [ ]:
correlation_matrix = features.corr()
plt.figure(figsize=(15, 10))
ax = sns.heatmap(
    correlation_matrix,
    vmax=1,
    square=True,
    annot=True,
    fmt='.2f',
    cmap='GnBu',
    cbar_kws={"shrink": .5},
    robust=True)
plt.title('Correlation Matrix of features', fontsize=8)
plt.show()

In [ ]:
list(features)

In [ ]:
# remove any high correlated variables
features = customers.drop(columns=['number_vmail_messages','total_day_charge','total_eve_charge','total_night_charge','total_intl_charge'])


# 2. Models Analysis

Four models have been constructed for application in the problem of this project, namely:

   * **Logistic Classification**
    
         Models the probability of the default class. The coefficients (Beta values b) of the logistic regression algorithm must be estimated from the training data. 
         
   * **Random Forest**
         
         Creates the forest with some number of trees.
         
   * **Artificial Neural Network (ANN)**
    
         It fits a sequence of weak learners on different weighted training data.
         
         

         
    

## 3. Steps in ML Analysis

1. Clean your data before analysis
2. Devide data into training set and test set
3. Train your model with training data set only
4. Check model performance for trained set and unknown test sen
5. Choose best model among them

### # 3.1 Logistic Regression

In [ ]:
customers = pd.read_csv('churn.csv')
customers = customers.drop(columns= ['area_code', 'phone_number', 'number_vmail_messages','total_day_charge','total_eve_charge','total_night_charge','total_intl_charge'])

In [ ]:
train, test= train_test_split(customers, test_size=0.2, random_state=1234)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 1234)

In [ ]:
classifier.fit(train.drop(columns='class'), train['class'])

In [ ]:
res = pd.DataFrame({'modelname':[None],'dataset':[None],'accuracy':[None], 'sensitivity':[None], 'specificity':[None], 'AUC':[None]})
y_pred_prob_train = classifier.predict_proba(train.drop(columns='class'))[:,1]
y_pred_train = classifier.predict(train.drop(columns='class'))
res.append(evalmodel(y_predicted=y_pred_train, y_pred_prob=y_pred_prob_train, y_actual=train['class'], modelname='Logistic', dataset='Train'))

In [ ]:
#Random forest
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 5, random_state = 42)

In [ ]:
classifier.fit(train.drop(columns='class'), train['class'])

In [ ]:
y_pred_prob_train = classifier.predict_proba(train.drop(columns='class'))[:,1]
y_pred_train = classifier.predict(train.drop(columns='class'))
res.append(evalmodel(y_predicted=y_pred_train, y_pred_prob=y_pred_prob_train, y_actual=train['class'], modelname='Random Forest', dataset='Train'))

In [ ]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(20,20),max_iter=1000, random_state=42)

In [ ]:
classifier.fit(train.drop(columns='class'), train['class'])

In [ ]:
y_pred_prob_train = classifier.predict_proba(train.drop(columns='class'))[:,1]
y_pred_train = classifier.predict(train.drop(columns='class'))
res.append(evalmodel(y_predicted=y_pred_train, y_pred_prob=y_pred_prob_train, y_actual=train['class'], modelname='ANN', dataset='Train'))

In [ ]:
y_pred_prob_test = classifier.predict_proba(test.drop(columns='class'))[:,1]
y_pred_test = classifier.predict(test.drop(columns='class'))
res.append(evalmodel(y_predicted=y_pred_test, y_pred_prob=y_pred_prob_test, y_actual=test['class'], modelname='ANN', dataset='Test'))

In [ ]:
## TO DO Test accuracy of TEST data